In [1]:
import pandas as pd
import sqlalchemy
from matplotlib import pyplot as plt
import datetime
import copy
import numpy as np

In [2]:
db = sqlalchemy.engine.url.URL(drivername='mysql',
                            host='127.0.0.1',
                            database='sherman_1',
                            query={'read_default_file': '~/.my.cnf', 'charset':'utf8'})
engine = sqlalchemy.create_engine(db)

In [3]:
df = pd.read_sql('''
    SELECT * FROM sherman_1.calls;
''' ,engine)

In [4]:
df['device_id'].nunique()

86

In [5]:
def get_incoming_outgoing_missed(df):
    uniq_device_id = list(set(df['device_id'].tolist()))
    df_output = pd.DataFrame()
    df_incoming = pd.DataFrame()
    df_outgoing = pd.DataFrame()
    df_missed = pd.DataFrame()
    for device_id in uniq_device_id:
        df_target = df[df['device_id']==device_id]
        df_target = df_target.sort_values(by = 'timestamp', ascending = False)
        
        df_output_temp = df_target[['device_id','timestamp']]
        
        past_type = df_target['call_type'].tolist()
        past_type = past_type[1:]+[None]
        future_type = df_target['call_type'].tolist()
        future_type = [None]+future_type[:-1]
        duration = df_target['call_duration'].tolist()
        duration = [None]+duration[:-1]
        
        df_target['past_type'] = past_type
        df_target['future_type'] = future_type
        df_target['duration'] = duration
        
        

        #incoming
        df_incoming_temp = df_target[(df_target['past_type']==1)&
                                   (df_target['call_type']==2)&
                                   (df_target['future_type']==4)]
        
        df_incoming_temp['incoming_time'] = df_incoming_temp['duration']
        df_incoming_temp = df_incoming_temp[['device_id','timestamp','incoming_time']]
        
        
        #outgoing
        df_outgoing_temp = df_target[(df_target['past_type']==3)&
                                   (df_target['call_type']==2)&
                                   (df_target['future_type']==4)]
        
        df_outgoing_temp['outgoing_time'] = df_outgoing_temp['duration']
        df_outgoing_temp = df_outgoing_temp[['device_id','timestamp','outgoing_time']]
        
        
        #missing
        df_missing_temp = df_target[(df_target['call_type']==4)&
                                   (df_target['past_type']!=2)]
        
        df_missing_temp['missing_time'] = df_missing_temp['duration']
        df_missing_temp = df_missing_temp[['device_id','timestamp','missing_time']]
        
        df_output_temp = df_output_temp.merge(df_incoming_temp, on = ['device_id','timestamp'], how = 'left')
        df_output_temp = df_output_temp.fillna(0)
        
        df_output_temp = df_output_temp.merge(df_outgoing_temp, on = ['device_id','timestamp'], how = 'left')
        df_output_temp = df_output_temp.fillna(0)
        
        df_output_temp = df_output_temp.merge(df_missing_temp, on = ['device_id','timestamp'], how = 'left')
        df_output_temp = df_output_temp.fillna(0)
        
        if df_output_temp.shape[0]>0:
            df_output = pd.concat([df_output, df_output_temp], ignore_index=True)
    
    return df_output

In [6]:
df_less = get_incoming_outgoing_missed(df)
df_less['date'] = pd.to_datetime(df_less['timestamp'],unit='ms').dt.date
df_less.head(10)

<ipython-input-5-a9169a7bde86>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_incoming_temp['incoming_time'] = df_incoming_temp['duration']
<ipython-input-5-a9169a7bde86>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_outgoing_temp['outgoing_time'] = df_outgoing_temp['duration']
<ipython-input-5-a9169a7bde86>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,device_id,timestamp,incoming_time,outgoing_time,missing_time,date
0,42476945-afd2-4b05-9a0c-2b2f44246c43,1.673465e+12,0.0,0.0,0.0,2023-01-11
1,42476945-afd2-4b05-9a0c-2b2f44246c43,1.673465e+12,0.0,0.0,0.0,2023-01-11
2,42476945-afd2-4b05-9a0c-2b2f44246c43,1.673454e+12,0.0,0.0,0.0,2023-01-11
3,42476945-afd2-4b05-9a0c-2b2f44246c43,1.673454e+12,248.0,0.0,0.0,2023-01-11
4,42476945-afd2-4b05-9a0c-2b2f44246c43,1.673454e+12,0.0,0.0,0.0,2023-01-11
5,42476945-afd2-4b05-9a0c-2b2f44246c43,1.673444e+12,0.0,0.0,0.0,2023-01-11
6,42476945-afd2-4b05-9a0c-2b2f44246c43,1.673443e+12,1106.0,0.0,0.0,2023-01-11
7,42476945-afd2-4b05-9a0c-2b2f44246c43,1.673443e+12,0.0,0.0,0.0,2023-01-11
8,42476945-afd2-4b05-9a0c-2b2f44246c43,1.673442e+12,0.0,0.0,0.0,2023-01-11
9,42476945-afd2-4b05-9a0c-2b2f44246c43,1.673442e+12,0.0,239.0,0.0,2023-01-11


In [7]:
df_less = df_less[['device_id','date','incoming_time','outgoing_time','missing_time']]
df_less['incoming_freq'] = df_less['incoming_time'].apply(lambda x: int(x>0))
df_less['outgoing_freq'] = df_less['outgoing_time'].apply(lambda x: int(x>0))
df_less['missing_freq'] = df_less['missing_time'].apply(lambda x: int(x>0))
df_output = df_less.groupby(['device_id','date']).agg(incoming_time = ('incoming_time', np.sum),
                                                      incoming_freq = ('incoming_freq',np.sum),
                                                      outgoing_time = ('outgoing_time', np.sum),
                                                      outgoing_freq = ('outgoing_freq',np.sum),
                                                      missing_time = ('missing_time', np.sum),
                                                      missing_freq = ('missing_freq',np.sum),
                                                     )
df_output = df_output.reset_index()
df_output['incoming_freq'] = df_output['incoming_freq'].apply(lambda x: int(x))
df_output['outgoing_freq'] = df_output['outgoing_freq'].apply(lambda x: int(x))
df_output['missing_freq'] = df_output['missing_freq'].apply(lambda x: int(x))
df_output['device_id'].nunique()

86

In [8]:
df_new = df_output.drop_duplicates(subset='device_id', keep="first")
df_new['device_id'].nunique()

86

In [9]:
df_id = pd.read_csv("/data/meditation/majid/tables/id.csv")
df_id = df_id.drop_duplicates('study_id')
df_id = df_id.drop('id', axis = 1)
df_id.columns = ['device_id','Study ID']
df_id

,device_id,Study ID
0,4a23c5dc-9a50-4d46-a12d-dd94ee98270f,8415477
1,f3845ba4-8702-42cf-a14f-051c46f66b61,15027959
2,ae02314b-a607-41b9-9507-28ffa1babed7,73386307
3,b460f89e-fd76-46ff-a960-989d74fcb66a,87785016
4,e5048fab-dc20-4d99-8cea-220a5a2750e0,14851895
...,...,...
70,780c9b18-72d3-4714-858d-3c91b3719155,21788465
71,a1118578-bbb4-47ef-b784-7d5bd09ad8b5,7389236
72,00e17b8a-4b0f-4230-bd26-3df2853e8916,69546645
73,029e57cb-fa0d-479d-abd2-9cb5745f219d,72690342


In [10]:
df_group = pd.read_csv('/data/meditation/majid/surveys/0.csv', skiprows=[1,2])[['Study ID','Group']]
df_group = df_id.merge(df_group, on = 'Study ID', how = 'left')
df_group['device_id'].nunique()

69

In [11]:
df_plot = df_plot.merge(df_group[['device_id','Group']], on = 'device_id', how = 'left')

NameError: name 'df_plot' is not defined

In [ ]:
df_plot['device_id'].nunique()

: 

In [ ]:
for group in ['Active Control', 'Experimental Condition', 'Passive Control']:
    df_target = df_plot[df_plot['Group']==group]
    print(df_target['device_id'].nunique())

: 